In [ ]:
import tensorflow as tf
import keras
import os

In [ ]:
gpus = tf.config.list_physical_devices(device_type='GPU')
print(gpus)
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
  except RuntimeError as e:
    print(e)

In [ ]:
(training_images, training_labels), \
(validation_images, validation_labels) = \
    tf.keras.datasets.mnist.load_data()

In [ ]:
def preprocess_image_input(input_images):
  # normalizing in [0, 1] range
  preprocessed_images = tf.pad(input_images, [[0, 0], [2, 2], [2, 2]]) / 255
  preprocessed_images = tf.expand_dims(preprocessed_images, axis=3, name=None)
  return preprocessed_images

train_X = preprocess_image_input(training_images)
valid_X = preprocess_image_input(validation_images)

In [ ]:
def create_model():
  Lenet = tf.keras.Sequential()
  Lenet.add(tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=(32,32,1)))
  Lenet.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2)))

  Lenet.add(tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='tanh'))
  Lenet.add(tf.keras.layers.AveragePooling2D(pool_size=(2, 2)))

  Lenet.add(tf.keras.layers.Conv2D(120, kernel_size=(5, 5), activation='tanh'))

  Lenet.add(tf.keras.layers.Flatten())
  Lenet.add(tf.keras.layers.Dense(84, activation='tanh'))
  Lenet.add(tf.keras.layers.Dense(10, activation='softmax'))

  return Lenet

'''
Define the model and compile it. 
Use Adam as the optimizer.
Use Sparse Categorical CrossEntropy as the loss function.
'''
def define_compile_model():
  Lenet = create_model()
  inputs = tf.keras.layers.Input(shape=(32,32,1), dtype=tf.float32)
  Lenet = Lenet(inputs)
  model = tf.keras.Model(inputs=inputs, outputs=Lenet)
  model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model

model = define_compile_model()
model.summary()

In [ ]:
EPOCHS = 1

history = model.fit(train_X, 
                    training_labels, 
                    epochs=EPOCHS, 
                    validation_data = (valid_X, validation_labels), 
                    batch_size=64)

In [ ]:
MODEL_PATH = "../models_data/lenet5/"
os.makedirs(MODEL_PATH, exist_ok=True)
model.save(MODEL_PATH+"lenet5_fp32_mnist.keras")

In [ ]:
model.evaluate(valid_X, validation_labels)
model.evaluate(valid_X[:2000], validation_labels[:2000])

In [ ]:
def representative_dataset():
  for images in tf.data.Dataset.from_tensor_slices(train_X).batch(1).take(1000):
    yield [images]

loaded_model = keras.saving.load_model("../models_data/lenet5/lenet5_fp32_mnist.keras")
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] 
converter.representative_dataset = representative_dataset

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_model = converter.convert()
with open('../models_data/lenet5/lenet5_uint8_mnist.tflite', 'wb') as f:
  f.write(tflite_quant_model)

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_quant_model = converter.convert()
with open('../models_data/lenet5/lenet5_int8_mnist.tflite', 'wb') as f:
  f.write(tflite_quant_model)
